```
This software is part of GPU Ocean. 

Copyright (C) 2017, 2018 SINTEF Digital
Copyright (C) 2017, 2018 Norwegian Meteorological Institute

This notebook implements the Rossby adjustment test case, as 
reported under Case B in Test Cases for Rotational Shallow-Water Schemes 
by Holm, Brodtkorb, Broström, Christensen and Sætra.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Case B: Rossby adjustment

This notebook runs the simulations for the Rossby adjustment problem. The results are visualized and plotted in the notebook `CaseB_RossbyAdjustment_Plotting.ipynb`.

## Some theory

In this notebook we investigate different properties with our numerical schemes related to geostrophic balance. Geostrophic balance represent steady state solutions where the pressure gradients are balanced by the Coriolis forces.

### Geostrophic Balance

The geostrophic equations in rotating shalow water is given by
$$ \frac{\partial u}{\partial t} - fv  = - \frac{1}{\rho} \frac{\partial p}{\partial x}, $$
$$ \frac{\partial v}{\partial t} + fu  = - \frac{1}{\rho} \frac{\partial p}{\partial y}. $$
By the assumption that the vertical velocity is negligible compared to the horizontal velocity, we integrate the equations vertically.
Using hydrostatic pressure $ p = \rho g (H+\eta) + p_{atm}$, we get an expression for the change in momentum as
$$ \frac{\partial hu}{\partial t} =  fhv - gh\frac{\partial \eta}{\partial x}, $$
$$ \frac{\partial hv}{\partial t} = -fhu - gh\frac{\partial \eta}{\partial y}. $$
At geostrophic balance, the steady state solution is described by $\frac{\partial hv}{\partial t} = \frac{\partial hu}{\partial t} = 0$.



###### Additional geostrophy equation
The following equation is related to the above, to describe the steady-state of the surface elevation:
$$\frac{\partial \eta}{\partial t} + hu\frac{\partial \eta }{\partial x } + hv \frac{\partial \eta }{\partial y} = 0 $$

## Rossby adjustment
Rossby adjustment is the process where an initial bump (of some sort), $\eta_0$, reaches the geostrophic balance steady-state $\bar{\eta}$. The size of the steady-state bump can be described by gravity $g$, lake-at-rest depth $H$ and Coriolis forces $f$ by the Klein-Gordon equation,
$$ -c_0^2 \nabla^2 \bar{\eta} + f^2 (\bar{\eta} - \eta_0) = 0, $$
or, more commonly,
$$  \nabla^2 \bar{\eta} -  \left( \frac{1}{a} \right)^2 (\bar{\eta} - \eta_0) = 0. $$
In this equation, $(1/a)^2$ is the Rossby radius. The constant $c_0^2 = gH$, meaning that $a^2 = gH/f^2$. 
The physical interpretation of the Rossby radius, $a$, is shown in the first figure below the initial imports.

Additionally, the steady-state solution should be reached within a period given by $\approx \pi/f$.


# Setting up the environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os, pyopencl, datetime, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
gpu_ctx = Common.CUDAContext()

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

## Evaluation of Klein-Gordon and the Geostrophic Balance

Given the equation for the geostrophic balance and the Klein-Gordon equations shown above, we approximate derivatives of the solution to investigate steady-state properties.

In [ ]:
"""
Returns d/dx(0.5*g*h*h) + fhv
"""
def geostrophicBalanceEta(eta, H, hu, hv, nx, ny, dx, dy, f, g):
    return geostrophicBalance(eta+H, hu, hv, nx, ny, dx, dy, f, g)

def geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells in input arrays
    A = hu*hv/h
    B = hu*hu/h
    C = 0.5*g*h*h
    D = -f*hv
    
    Ay = np.zeros_like(A)
    Bx = np.zeros_like(B)
    Cx = np.zeros_like(C)
    
    Ay[1:-1,:] = (A[:-2,:] - A[2:,:])/(2*dy)
    Bx[:, 1:-1] = (B[:,:-2] - B[:,2:])/(2*dx)
    Cx[:, 1:-1] = (C[:,:-2] - C[:,2:])/(2*dx)
    
    geoBalance = Cx - D
    return geoBalance

def geostrophicBalanceStaggered(eta, H, hu_s, hv_s, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
       
    h = eta + H
    hu = 0.5*(hu_s[:, :-1] + hu_s[:, 1:])
    hv = 0.5*(hv_s[:-1, :] + hv_s[1:, :])
    return geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g )

In [ ]:
def kleinGordonBalance(eta, H, eta_0, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
    c = np.sqrt(g*H)
    a = c/f
    constant = 1/(a*a)
    print "a: , const: ", a, constant
    print "dx, dy:", dx, dy
    print "max eta-eta_0: ", np.max(eta - eta_0)
    
    eta_xx = np.zeros_like(eta)
    eta_yy = np.zeros_like(eta)
    laplace_eta = np.zeros_like(eta) 
        
    eta_yy[1:-1, :] = (eta[:-2,:] -2*eta[1:-1,:] + eta[2:, :])/(dy*dy)
    eta_xx[:, 1:-1] = (eta[:, :-2] - 2*eta[:, 1:-1] + eta[:, 2:])/(dx*dx)
    laplace_eta = eta_xx + eta_yy
    print "max Laplacean: ", np.max(laplace_eta)
    
    kg = laplace_eta - constant*(eta - eta_0)
    return kg
    
    

# Rossby adjustment on flat bottom

The area of interest is simply the neighbouring region around the initial bump (where  $\bar{\eta} \neq 0$ at steady state). We will therefore use a domain with open boundary conditions to allow gravity waves to escape the domain.

In [ ]:
# Common parameters from Kai

nx = 800
ny = 1000

dx = 50000
dy = 50000

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
H0 = 1000.0
A = 0.0 # A is diffusion coefficient multiplied by depth.

wind = Common.WindStressParams(type=99)

# Using an open boundary
sponge = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=sponge)

ghosts = [10, 10, 10, 10]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
validDomain =  [10, 10, 10, 10]
cutoff = 10


# Rossby adjustment parameters:
c0_squared = g*H0
a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

#T = 300#sub_T = 4000.0

T = 38
sub_T = 300*dt
make_netCDF = True
def get_sub_T(i):
    if i < 21:
        return sub_T
    else:
        return sub_T*40

totSimTime = 0
for i in range(T):
    totSimTime = totSimTime + get_sub_T(i)
print "Tot sim time: ", totSimTime-sub_T

interestingDomain = 250

paramSet = "Kai"
msg = "Using variables from " + paramSet

# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

##### Creating initial conditions

Staggered and unstaggered grids are placed so that cell centers are on the same place. Keep therefore in mind that the velocities are defined at different positions for the two different grid types!

In [ ]:
def initialConditions(eta0, nx, ny, dx, dy, halo_x, halo_y):
    if paramSet == "Haavard":
        ghosts = [halo_y, halo_x, halo_y, halo_x]
        addCentralBump(eta0, nx, ny, dx/5, dy/5, ghosts)
        #addCentralDamBreakStep(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        #addCentralDamBreakSmooth(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        return
    print "Making initial conditions"
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    y_center1 = dy*(ny-100)/2.0
    y_center2 = dy*(ny+100)/2.0

    for j in range(-halo_y, ny+halo_y):
        for i in range(-halo_x, nx+halo_x):
            x = dx*i - x_center 
            y = dy*j - y_center 
            y1 = dy*j - y_center1
            y2 = dy*j - y_center2

            # Initial smooth step
            inirad = np.sqrt(x**2 + y**2)
            inirad1 = np.sqrt(x**2 + y1**2)
            inirad2 = np.sqrt(x**2 + y2**2)
            L = 15*dx
            D = 50*dx
            etaamp = 0.2
            
            # Add the the initial condition to the provided array
            #eta0[j+1, i+1] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            eta0[j+halo_y, i+halo_x] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            

## Executing simulators

In [ ]:
## Forward-Backward-Linear

fbl_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

fbl_eta0 = np.zeros(dataShape, dtype=np.float32);
fbl_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
fbl_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);

initialConditions(fbl_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'fbl_sim' in globals():
    fbl_sim.cleanUp()
reload(FBL)
fbl_sim = FBL.FBL(gpu_ctx, \
                  fbl_h0, fbl_eta0, fbl_u0, fbl_v0, \
                  nx, ny, \
                  dx, dy, dt, \
                  g, f, r, \
                  wind_stress=wind, \
                  boundary_conditions=boundaryConditions, \
                  write_netcdf=make_netCDF )

print "Shape eta0", fbl_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]].shape
print "Shape u0  ", fbl_u0[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1].shape
print "Shape v0  ", fbl_v0[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]].shape

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                fbl_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                                fbl_u0[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                                fbl_v0[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]])

print "FBL\n" + msg
def animate(i):
    if (i>0):
        t = fbl_sim.step(get_sub_T(i))
    else:
        t = 0.0
    eta1, u1, v1 = fbl_sim.download()

    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                 u1[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                 v1[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]]);
    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%5 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1 + H0)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_FBL.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
fbl_sim.cleanUp()


In [ ]:
#Centered in time, centered in space

ctcs_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
ctcs_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
ctcs_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

initialConditions(ctcs_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#fig = plt.figure(figsize=(5,5))
#plt.imshow(ctcs_eta0)
#plt.colorbar()
#fig = plt.figure(figsize=(4,4))
#plt.plot(ctcs_eta0[dataShape[0]/2, 200:600])
#lsdgikhlghjksd


#Initialize simulator
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()
reload(CTCS)
ctcs_sim = CTCS.CTCS(gpu_ctx, \
                     ctcs_h0, ctcs_eta0, ctcs_u0, ctcs_v0, \
                     nx, ny, \
                     dx, dy, dt, \
                     g, f, r, A, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions, \
                     write_netcdf=make_netCDF)

print "Shape eta0: ", ctcs_eta0[ghosts[3]:-ghosts[1], ghosts[2]:-ghosts[0]].shape
print "Shape u0:   ", ctcs_u0[ghosts[3]:-ghosts[1], ghosts[2]-1:-ghosts[0]+1].shape
print "Shape v0:   ", ctcs_v0[ghosts[3]-1:-ghosts[1]+1, ghosts[2]:-ghosts[0]].shape
    
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                ctcs_eta0[ghosts[3]:-ghosts[1], ghosts[2]:-ghosts[0]],
                                ctcs_u0[ghosts[3]:-ghosts[1], ghosts[2]-1:-ghosts[0]+1],
                                ctcs_v0[ghosts[3]-1:-ghosts[1]+1, ghosts[2]:-ghosts[0]])

print "CTCS\n" + msg
def animate(i):
    if (i>0):
        t = ctcs_sim.step(get_sub_T(i))
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()

    plotter.plot(eta1[ghosts[3]:-ghosts[1], ghosts[2]:-ghosts[0]],
                 u1[ghosts[3]:-ghosts[1], ghosts[2]-1:-ghosts[0]+1],
                 v1[ghosts[3]-1:-ghosts[1]+1, ghosts[2]:-ghosts[0]])
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%5 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1 + H0)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
ctcs_sim.cleanUp()

In [ ]:
#Coriolis well balanced reconstruction scheme

cdklm_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
cdklm_u0   = np.zeros(dataShape, dtype=np.float32, order='C');
cdklm_v0   = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*H0
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

initialConditions(cdklm_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'cdklm_sim' in globals():
    cdklm_sim.cleanUp()
reload(CDKLM16)
cdklm_sim = CDKLM16.CDKLM16(gpu_ctx, \
                            cdklm_eta0, cdklm_u0, cdklm_v0, \
                            Hi, \
                            nx, ny, \
                            dx, dy, dt, \
                            g, f, r, \
                            wind_stress=wind, \
                            boundary_conditions=boundaryConditions, \
                            reportGeostrophicEquilibrium=False, \
                            write_netcdf=make_netCDF)



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                cdklm_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                cdklm_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                cdklm_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

#Kx = np.zeros(T)
#Ly = np.zeros(T)
#uxpvy = np.zeros(T)

print "CDKLM\n" + msg
def animate(i):
    if (i>0):
        t = cdklm_sim.step(get_sub_T(i))
    else:
        t = 0.0
    eta1, u1, v1 = cdklm_sim.download()
    #uxpvy[i], Kx[i], Ly[i] = cdklm_sim.downloadGeoEqNorm()
    
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                   u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                   v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%5 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
cdklm_sim.cleanUp()


In [ ]:
# Kurgonov-Petrove scheme from 2007

kp07_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
kp07_u0   = np.zeros(dataShape, dtype=np.float32, order='C');
kp07_v0   = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*H0
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

initialConditions(kp07_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()
reload(KP07)
kp07_sim = KP07.KP07(gpu_ctx, \
                     kp07_eta0, Hi, kp07_u0, kp07_v0, \
                     nx, ny, \
                     dx, dy, dt, \
                     g, f, r, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions, \
                     write_netcdf=make_netCDF)



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                kp07_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                kp07_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                kp07_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])


print "KP07\n" + msg
def animate(i):
    if (i>0):
        t = kp07_sim.step(get_sub_T(i))
    else:
        t = 0.0
    eta1, u1, v1 = kp07_sim.download()
    
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                   u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                   v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()